<a href="https://colab.research.google.com/github/yuridadt/play_with_keras/blob/master/Predicting_Skin_Cancer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predicting Skin Cancer

### Подготовка фотографий для загрузки на гугл диск и обработки керасом

In [0]:
import glob
import os
import random
from PIL import Image

In [0]:
#уменьшение размера картинок, у которых ширина < 1200 
def scale_image(input_image_path,
                output_image_path,
                width=None,
                height=None
                ):
    original_image = Image.open(input_image_path)
    w, h = original_image.size
    print('The original image size is {wide} wide x {height} '
          'high'.format(wide=w, height=h))
    if w <= 1200:
        original_image.save(output_image_path)
    else:
        if width and height:
            max_size = (width, height)
        elif width:
            max_size = (width, h)
        elif height:
            max_size = (w, height)
        else:
            # No width or height specified
            raise RuntimeError('Width or height required!')

        original_image.thumbnail(max_size, Image.ANTIALIAS)
        original_image.save(output_image_path)

        scaled_image = Image.open(output_image_path)
        width, height = scaled_image.size
        print('The scaled image size is {wide} wide x {height} '
              'high'.format(wide=width, height=height))

#перенесение картинок
def file_remover (dir_from, dir_to, count):
    for i in range(count):
        file_name = random.choice(os.listdir(dir_from))
        os.replace(dir_from + '\\' + file_name, dir_to + '\\' + file_name)
        i += 1

In [0]:
file_list_benign = glob.glob("isic_benign_train/*.jpg")
for file in file_list_benign:
    file_changed = file[:-16]+'ben_'+file[-16:]
    scale_image(input_image_path=file, output_image_path=file_changed, width=1024)

file_list_malignant = glob.glob("isic_malignant_train/*.jpg")
for file in file_list_malignant:
    file_changed = file[:-16] + 'mal_' + file[-16:]
    scale_image(input_image_path=file, output_image_path=file_changed, width=1024)

file_remover('\deep learn\isic_benign_train',
            '\deep learn\isic_test',
            300)

file_remover('\deep learn\isic_malignant_train',
            'deep learn\isic_test',
            300)


Далее картинки были добавлены в размеченные папки с классами, а затем перемещены с локального компьютера в гугл диск

In [26]:
#import content from google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Keras

In [1]:
# Importing Keras packages
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

Using TensorFlow backend.


In [2]:
#initializing the CNN
classifier = Sequential()

In [3]:
#Adding the Convolution Layer
classifier.add(Convolution2D(32, 3, 3, input_shape = (32, 32, 3), activation = "relu"))
#Adding the Pooling Layer
classifier.add(MaxPooling2D(pool_size=(3, 3)))
#Flattening the layer
classifier.add(Flatten())
# Full Connection
classifier.add(Dense(output_dim = 128, activation = 'relu'))
classifier.add(Dense(output_dim = 1, activation = 'sigmoid'))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(32, 32, 3..., activation="relu")`
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=128)`
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="sigmoid", units=1)`
  


In [4]:
#Compiling the CNN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
train_data_dir = 'drive/My Drive/deep learning/isic_train'
validation_data_dir = 'drive/My Drive/deep learning/isic_test'
img_width, img_height = 32, 32

In [30]:
#Fitting our CNN to the image dataset
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
                    rescale=1./255,
                    shear_range=0.2,
                    zoom_range=0.2,
                    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory(train_data_dir,
                                                 target_size = (img_width, img_height),
                                                 batch_size=10,
                                                 class_mode='binary')
test_set = test_datagen.flow_from_directory(validation_data_dir,
                                            target_size=(img_width, img_height),
                                            batch_size=32,
                                            class_mode='binary')
classifier.fit_generator(training_set,
                         steps_per_epoch=1255,
                         epochs=25,
                         validation_data= test_set, 
                         validation_steps=400)

Found 1265 images belonging to 2 classes.
Found 400 images belonging to 2 classes.
Epoch 1/25
1255/1255 [==============================] - 691s 551ms/step - loss: 0.4545 - acc: 0.7825 - val_loss: 0.1292 - val_acc: 0.9625
Epoch 2/25
1255/1255 [==============================] - 313s 249ms/step - loss: 0.3138 - acc: 0.8578 - val_loss: 0.1548 - val_acc: 0.9373
Epoch 3/25
1255/1255 [==============================] - 308s 246ms/step - loss: 0.2598 - acc: 0.8855 - val_loss: 0.2036 - val_acc: 0.9200
Epoch 4/25
1255/1255 [==============================] - 308s 245ms/step - loss: 0.2172 - acc: 0.9068 - val_loss: 0.1988 - val_acc: 0.9224
Epoch 5/25
1255/1255 [==============================] - 303s 242ms/step - loss: 0.1712 - acc: 0.9310 - val_loss: 0.1654 - val_acc: 0.9351
Epoch 6/25
1255/1255 [==============================] - 305s 243ms/step - loss: 0.1442 - acc: 0.9427 - val_loss: 0.2555 - val_acc: 0.9151
Epoch 7/25
1255/1255 [==============================] - 313s 249ms/step - loss: 0.1185 - 

### Результаты работы модели: 
 Found 1265 images belonging to 2 classes.
Found 400 images belonging to 2 classes.
Epoch 1/25
1255/1255 [==============================] - 691s 551ms/step - loss: 0.4545 - acc: 0.7825 - val_loss: 0.1292 - val_acc: 0.9625
Epoch 2/25
1255/1255 [==============================] - 313s 249ms/step - loss: 0.3138 - acc: 0.8578 - val_loss: 0.1548 - val_acc: 0.9373
Epoch 3/25
1255/1255 [==============================] - 308s 246ms/step - loss: 0.2598 - acc: 0.8855 - val_loss: 0.2036 - val_acc: 0.9200
Epoch 4/25
1255/1255 [==============================] - 308s 245ms/step - loss: 0.2172 - acc: 0.9068 - val_loss: 0.1988 - val_acc: 0.9224
Epoch 5/25
1255/1255 [==============================] - 303s 242ms/step - loss: 0.1712 - acc: 0.9310 - val_loss: 0.1654 - val_acc: 0.9351
Epoch 6/25
1255/1255 [==============================] - 305s 243ms/step - loss: 0.1442 - acc: 0.9427 - val_loss: 0.2555 - val_acc: 0.9151
Epoch 7/25
1255/1255 [==============================] - 313s 249ms/step - loss: 0.1185 - acc: 0.9545 - val_loss: 0.3036 - val_acc: 0.9148
Epoch 8/25
1255/1255 [==============================] - 304s 243ms/step - loss: 0.1016 - acc: 0.9626 - val_loss: 0.2757 - val_acc: 0.9226
Epoch 9/25
1255/1255 [==============================] - 309s 246ms/step - loss: 0.0919 - acc: 0.9664 - val_loss: 0.2533 - val_acc: 0.9349
Epoch 10/25
1255/1255 [==============================] - 308s 246ms/step - loss: 0.0736 - acc: 0.9746 - val_loss: 0.3598 - val_acc: 0.8972
Epoch 11/25
1255/1255 [==============================] - 305s 243ms/step - loss: 0.0663 - acc: 0.9764 - val_loss: 0.2852 - val_acc: 0.9401
Epoch 12/25
1255/1255 [==============================] - 303s 242ms/step - loss: 0.0548 - acc: 0.9822 - val_loss: 0.3915 - val_acc: 0.9002
Epoch 13/25
1255/1255 [==============================] - 305s 243ms/step - loss: 0.0558 - acc: 0.9802 - val_loss: 0.3263 - val_acc: 0.9149
Epoch 14/25
1255/1255 [==============================] - 306s 244ms/step - loss: 0.0437 - acc: 0.9847 - val_loss: 0.3064 - val_acc: 0.9377
Epoch 15/25
1255/1255 [==============================] - 307s 245ms/step - loss: 0.0545 - acc: 0.9793 - val_loss: 0.4244 - val_acc: 0.9098
Epoch 16/25
1255/1255 [==============================] - 305s 243ms/step - loss: 0.0503 - acc: 0.9816 - val_loss: 0.3448 - val_acc: 0.9325
Epoch 17/25
1255/1255 [==============================] - 306s 244ms/step - loss: 0.0360 - acc: 0.9869 - val_loss: 0.4686 - val_acc: 0.9152
Epoch 18/25
1255/1255 [==============================] - 306s 244ms/step - loss: 0.0454 - acc: 0.9816 - val_loss: 0.4101 - val_acc: 0.9175
Epoch 19/25
1255/1255 [==============================] - 304s 243ms/step - loss: 0.0342 - acc: 0.9865 - val_loss: 0.4278 - val_acc: 0.9300
Epoch 20/25
1255/1255 [==============================] - 305s 243ms/step - loss: 0.0405 - acc: 0.9853 - val_loss: 0.3840 - val_acc: 0.9199
Epoch 21/25
1255/1255 [==============================] - 306s 244ms/step - loss: 0.0324 - acc: 0.9861 - val_loss: 0.4150 - val_acc: 0.9194
Epoch 22/25
1255/1255 [==============================] - 303s 242ms/step - loss: 0.0273 - acc: 0.9885 - val_loss: 0.4264 - val_acc: 0.9154
Epoch 23/25
1255/1255 [==============================] - 307s 245ms/step - loss: 0.0347 - acc: 0.9851 - val_loss: 0.4025 - val_acc: 0.9202
Epoch 24/25
1255/1255 [==============================] - 310s 247ms/step - loss: 0.0290 - acc: 0.9871 - val_loss: 0.4598 - val_acc: 0.9174
Epoch 25/25
1255/1255 [==============================] - 310s 247ms/step - loss: 0.0373 - acc: 0.9860 - val_loss: 0.6057 - val_acc: 0.8922
<keras.callbacks.History at 0x7fb10f392f60>